# Instant Gratification - Hugging Face Version

## Your first LLM Project using Hugging Face!

This notebook has been modified to use Hugging Face's API instead of OpenAI.

In [ ]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from transformers import pipeline
from huggingface_hub import InferenceClient

# Connecting to Hugging Face

You'll need to set up your Hugging Face API key in your .env file:
HUGGINGFACE_API_KEY=your_api_key_here

In [ ]:
# Load environment variables

load_dotenv(override=True)
api_key = os.getenv('HUGGINGFACE_API_KEY')

# Check the key
if not api_key:
    print("No Hugging Face API key found - please add it to your .env file")
else:
    print("API key found!")

# Initialize Hugging Face client
client = InferenceClient(token=api_key)

In [ ]:
# Quick test of Hugging Face API
message = "Hello, this is my first message to you! Hi!"

# Using Mistral model as an example - you can change this to any other model you prefer
response = client.text_generation(
    prompt=message,
    model="mistralai/Mistral-7B-Instruct-v0.1",
    max_new_tokens=100
)

print(response)

In [ ]:
# A class to represent a Webpage
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
def create_prompt(website):
    """Create a prompt for the model that includes instructions and website content"""
    return f"""Below is the content of a website titled '{website.title}'. 
    Please provide a short, clear summary of the main content. 
    If there are any news or announcements, include those too.
    
    Content:
    {website.text}
    
    Summary:"""

In [ ]:
def summarize(url):
    """Summarize a website using Hugging Face's API"""
    website = Website(url)
    prompt = create_prompt(website)
    
    # Using Mistral model - you can change this to other models
    response = client.text_generation(
        prompt=prompt,
        model="mistralai/Mistral-7B-Instruct-v0.1",
        max_new_tokens=200,
        temperature=0.7
    )
    
    return response

In [ ]:
def display_summary(url):
    """Display the summary in markdown format"""
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
# Try it out!
display_summary("https://edwarddonner.com")